In [88]:
import pickle
import numpy as np
import pandas as pd
import os

import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation
%matplotlib inline 
sns.set(color_codes=True)

from datetime import datetime
from dateutil.relativedelta import relativedelta

In [89]:
with open("BondPrice_Prediction_Model.pkl", 'rb') as file:  
    model = pickle.load(file)

In [90]:
merp_df = pd.read_csv ('C:\\Users\\richi\\OneDrive\\Desktop\\Finance\\BPAMERP20201130-TEST.csv')
print (merp_df)

mers_df = pd.read_csv ('C:\\Users\\richi\\OneDrive\\Desktop\\Finance\\BPAMERS20201130-TEST.csv')
print (mers_df)

  STOCK CODE     ISIN CODE                                         STOCK NAME  \
0   VN140167  MYBVN1401675                     TELEKOM IMTN 4.738% 27.06.2024   
1   VO180016  MYBVO1800163  EDRA ENERGY IMTN 6.030% 05.01.2029 - Tranche N...   
2   PU110040  MYBPU1100407                          BESRAYA 5.360% 28.07.2028   
3   VS180380  MYBVS1803804              DHSB IMTN Series 10 6.650% 21.06.2033   
4   VZ170398  MYBVZ1703983                  MAYBANKISLM IMTN 4.950% PERPETUAL   
5   VN190102  MYBVN1901021         PASB IMTN 4.150% 04.06.2029 - Issue No. 15   
6   UI180018  MYBUI1800183             TOYOTA CAP MTN 1826D 24.1.2023 - MTN 2   
7   UN170079  MYBUN1700797  CIMBBANK MTN 3652D 18.5.2027 - SERIES 1 TRANCHE 3   
8   VT080773  MYBVT0807739                             JEV IMTN 0% 10.11.2023   
9   VI170138  MYBVI1701389  ALDZAHAB ABS-IMTN 29.03.2022(CLASS A TRANCHE 3...   

  VALUE DATE  EVAL UPPER THRESHOLD YIELD  EVAL MID YIELD  \
0   1-Dec-20                       2.443        

In [91]:
merp_df.shape

(10, 19)

In [92]:
mers_df.shape

(10, 33)

In [93]:
df = merp_df.merge(mers_df,on=['STOCK CODE','ISIN CODE','STOCK NAME'],how='inner')
print(df.shape)

(10, 49)


### Data Preparation

In [94]:
for col in ['VALUE DATE','PREVIOUS PAYMENT DATE','ISSUE DATE','MATURITY DATE','NEXT PAYMENT DATE']:
    df[col] = pd.to_datetime(df[col])

In [95]:
df['PREVIOUS PAYMENT DATE for calculation'] = df['PREVIOUS PAYMENT DATE'].copy()
df.loc[df['PREVIOUS PAYMENT DATE for calculation'].isnull(),'PREVIOUS PAYMENT DATE for calculation']=df.loc[df['PREVIOUS PAYMENT DATE for calculation'].isnull()]['ISSUE DATE']

In [96]:
df["Value-Prev"] = df['VALUE DATE'] - df['PREVIOUS PAYMENT DATE for calculation']
df["Value-Prev"] = df["Value-Prev"].astype(str).str.replace(" days","").astype(int)
df["Payment Days diff"] = df['NEXT PAYMENT DATE'] - df['PREVIOUS PAYMENT DATE']
df["Payment Days diff"] = df["Payment Days diff"].astype(str).str.replace(" days","").astype(int)

In [97]:
thisdict = {
  "ACTACT": 1,
  "ACT365": 2,
}

df['DAY COUNT BASIS (INT)']= df['DAY COUNT BASIS'].map(thisdict)

In [98]:
nominal = 5000000

df["ACCRUED INTEREST 1"] = nominal*(df['NEXT COUPON RATE']/100)*(df["Value-Prev"]/df["Payment Days diff"])
df["ACCRUED INTEREST 2"] = nominal*(df['NEXT COUPON RATE']/100)*(df["Value-Prev"]/365)

df['ACCRUED INTEREST'] = df["ACCRUED INTEREST 1"].copy()
df.loc[df['DAY COUNT BASIS (INT)']==2,'ACCRUED INTEREST'] = df.loc[df['DAY COUNT BASIS (INT)']==2]["ACCRUED INTEREST 2"]

In [99]:
df["TERM OF MATURITY"]= (df['MATURITY DATE'] - df['VALUE DATE'])
df["TERM OF MATURITY"] = df["TERM OF MATURITY"].astype(str).str.replace(" days","").astype(int)
df["TERM OF MATURITY"] = df["TERM OF MATURITY"]/365

### Bond Price Prediction in Dec'2020

In [100]:
X_col = ['EVAL MID YIELD','EVAL MID PRICE','ACCRUED INTEREST',"TERM OF MATURITY",'MODIFIED DURATION']
df['PREDICTED DECEMBER BOND PRICE'] = model.predict(df[X_col])

In [101]:
df[['STOCK CODE','ISIN CODE','STOCK NAME','EVAL MID PRICE','PREDICTED DECEMBER BOND PRICE']]

,STOCK CODE,ISIN CODE,STOCK NAME,EVAL MID PRICE,PREDICTED DECEMBER BOND PRICE
0,VN140167,MYBVN1401675,TELEKOM IMTN 4.738% 27.06.2024,107.923,107.457669
1,VO180016,MYBVO1800163,EDRA ENERGY IMTN 6.030% 05.01.2029 - Tranche N...,116.587,116.335189
2,PU110040,MYBPU1100407,BESRAYA 5.360% 28.07.2028,112.588,112.618872
3,VS180380,MYBVS1803804,DHSB IMTN Series 10 6.650% 21.06.2033,111.121,112.215004
4,VZ170398,MYBVZ1703983,MAYBANKISLM IMTN 4.950% PERPETUAL,103.253,103.299934
5,VN190102,MYBVN1901021,PASB IMTN 4.150% 04.06.2029 - Issue No. 15,107.690,107.985974
6,UI180018,MYBUI1800183,TOYOTA CAP MTN 1826D 24.1.2023 - MTN 2,103.922,103.711859
7,UN170079,MYBUN1700797,CIMBBANK MTN 3652D 18.5.2027 - SERIES 1 TRANCHE 3,110.162,109.936663
8,VT080773,MYBVT0807739,JEV IMTN 0% 10.11.2023,118.937,117.712659
9,VI170138,MYBVI1701389,ALDZAHAB ABS-IMTN 29.03.2022(CLASS A TRANCHE 3...,102.962,102.641150
